In [1]:
%load_ext autoreload
%autoreload now

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"
# os.environ["WORLD_SIZE"] = "1"

# Download and convert data

In [3]:
from tqdm import tqdm

from document_segmentation.pagexml.annotations.renate_analysis import RenateAnalysis
from document_segmentation.settings import RENATE_ANALYSIS_DIR

N = None

RENATE_ANALYSIS_DIR.mkdir(parents=True, exist_ok=True)

sheet = RenateAnalysis()


existing_docs = {
    path.stem for path in RENATE_ANALYSIS_DIR.glob("Globdoc_*.json") if path.is_file()
}

for document in tqdm(
    sheet.to_documents(n=N, skip_ids=existing_docs),
    total=(N or len(sheet)) - len(existing_docs),
    desc="Writing documents",
    unit="doc",
):
    document_file = RENATE_ANALYSIS_DIR / f"{document.id}.json"

    with document_file.open("xt") as f:
        f.write(document.model_dump_json())
        f.write("\n")

Writing documents: 0doc [00:00, ?doc/s]


In [4]:
import logging

from tqdm import tqdm

from document_segmentation.pagexml.annotations.renate_analysis import RenateAnalysisInv
from document_segmentation.settings import RENATE_ANALYSIS_DIR, RENATE_ANALYSIS_SHEETS

N = None


sheet = RenateAnalysisInv(RENATE_ANALYSIS_SHEETS[0])  # TODO: use both sheets

for document in tqdm(
    sheet.to_documents(n=N), desc="Writing documents", unit="doc", total=26
):
    document_file = RENATE_ANALYSIS_DIR / f"{document.id}.json"

    if document_file.exists():
        logging.info(f"Document {document.id} already exists, skipping")
    else:
        with document_file.open("xt") as f:
            f.write(document.model_dump_json())
            f.write("\n")

Writing documents:   0%|          | 0/26 [00:00<?, ?doc/s]

Writing documents: 100%|██████████| 26/26 [00:10<00:00,  2.54doc/s]


# Load Data

In [5]:
%autoreload now

In [6]:
TRAINING_DATA = 0.8

In [7]:
from document_segmentation.model.dataset import DocumentDataset

dataset: DocumentDataset = DocumentDataset.from_dir(RENATE_ANALYSIS_DIR)
dataset.shuffle()

len(dataset)

Reading JSON files: 100%|██████████| 104/104 [00:00<00:00, 179.17file/s]


104

In [8]:
dataset._class_counts()

Counter({<Label.IN: 1>: 1907,
         <Label.BEGIN: 0>: 104,
         <Label.END: 2>: 100,
         <Label.OUT: 3>: 73})

In [9]:
dataset.class_weights()

[0.9904761904761905,
 0.05450733752620545,
 1.0297029702970297,
 1.4054054054054055]

In [10]:
training_data, test_data = dataset.split(TRAINING_DATA)

In [11]:
training_data._class_counts()

Counter({<Label.IN: 1>: 1642,
         <Label.BEGIN: 0>: 83,
         <Label.END: 2>: 80,
         <Label.OUT: 3>: 68})

In [12]:
test_data._class_counts()

Counter({<Label.IN: 1>: 265,
         <Label.BEGIN: 0>: 21,
         <Label.END: 2>: 20,
         <Label.OUT: 3>: 5})

# Train Model

In [13]:
import torch

BATCH_SIZE = 64
EPOCHS = 5
WEIGHTS = torch.Tensor(dataset.class_weights())  # For an imbalanced dataset

In [14]:
%autoreload now

In [15]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

In [16]:
tagger._device

'mps'

In [17]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbeddingSentenceTransformer(
      (_transformer_model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
      )
      (_region_type): Embedding(9, 16)
      (_linear): Linear(in_features=784, out_features=512, bias=True)
    )
    (_rnn): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (_linear): Linear(in_features=512, out_features=256, bias=True)
  )
  (_rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (_linear): Linear(in_features=512, out_features=4, bias=True)
  (_soft

In [18]:
tagger.train_(training_data, EPOCHS, BATCH_SIZE, WEIGHTS.to(tagger._device))

Training: 100%|██████████| 95/95 [01:43<00:00,  1.09s/batch]


[Loss:	0.860]


Training: 100%|██████████| 95/95 [00:05<00:00, 17.30batch/s]


[Loss:	0.594]


Training: 100%|██████████| 95/95 [00:05<00:00, 17.59batch/s]


[Loss:	0.532]


Training: 100%|██████████| 95/95 [00:05<00:00, 17.30batch/s]


[Loss:	0.519]


Training: 100%|██████████| 95/95 [00:05<00:00, 17.27batch/s]

[Loss:	0.517]


In [24]:
with open("page_sequence_tagger.pt", "xb") as f:
    torch.save(tagger, f)

# Evaluate Model

In [19]:
import csv
import sys

from torcheval.metrics import (
    MulticlassAccuracy,
    MulticlassF1Score,
    MulticlassPrecision,
    MulticlassRecall,
)
from tqdm import tqdm

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout,
    fieldnames=("Predicted", "Actual", "Page ID", "Text", "Scores"),
    delimiter="\t",
)

writer.writeheader()

accuracy = MulticlassAccuracy(num_classes=len(Label))
precision = MulticlassPrecision(average=None, num_classes=len(Label))
recall = MulticlassRecall(average=None, num_classes=len(Label))
f1_score = MulticlassF1Score(average=None, num_classes=len(Label))

for batch in tqdm(
    test_data.batches(BATCH_SIZE), total=test_data.n_batches(BATCH_SIZE), unit="batch"
):
    predicted = tagger(batch)
    labels = batch.labels()

    _labels = torch.Tensor([label.value for label in labels]).to(int)
    accuracy.update(predicted, _labels)
    precision.update(predicted, _labels)
    recall.update(predicted, _labels)
    f1_score.update(predicted, _labels)

    for page, pred, label in zip(batch.pages, predicted, labels):
        pred_label = Label(pred.argmax().item())
        writer.writerow(
            {
                "Predicted": pred_label.name,
                "Actual": label.name,
                "Page ID": page.doc_id,
                "Text": page.text(delimiter="; ")[:50],
                "Scores": str(pred.tolist()),
            }
        )

Predicted	Actual	Page ID	Text	Scores


  5%|▍         | 1/21 [00:00<00:15,  1.29batch/s]

BEGIN	BEGIN	NL-HaNA_1.04.02_2334_0029.jpg	N„o 1. Originele missive door en aan als boven; „ 	[0.9976346492767334, 0.0014405688270926476, 0.0003242988314013928, 0.0006004812312312424]
IN	IN	NL-HaNA_1.04.02_2334_0030.jpg	bodemapart; inhandigt.; Cognossement; Onkost reek„	[0.0210682712495327, 0.9577849507331848, 0.01492651179432869, 0.0062202634289860725]
END	END	NL-HaNA_1.04.02_2334_0031.jpg	stuagtelijst; lijste de Maarte: haarteken; den sch	[0.0026382452342659235, 0.002028851304203272, 0.9950413107872009, 0.0002916694793384522]


 10%|▉         | 2/21 [00:01<00:11,  1.60batch/s]

BEGIN	BEGIN	NL-HaNA_1.04.02_2542_0114.jpg	Van Mallabaar onder dato; 9. stux te weeten; 6. ma	[0.9847745299339294, 0.0008813795284368098, 0.014121505431830883, 0.00022256876400206238]


...

BEGIN	BEGIN	NL-HaNA_1.04.02_1072_0179.jpg	Legatten ende clein vaestngen nooducs; voor Banda.	[0.9972061514854431, 0.0016327535267919302, 0.0003721415705513209, 0.0007889929111115634]
IN	IN	NL-HaNA_1.04.02_1072_0180.jpg	Jacht van Banda, van daer bij oostlycke; monssor n	[0.011204724200069904, 0.9797572493553162, 0.004032177850604057, 0.005005925428122282]
IN	IN	NL-HaNA_1.04.02_1072_0181.jpg	insolentien vande Bandanesen.; Moordadus feit van 	[0.007464708760380745, 0.9854023456573486, 0.0032674826215952635, 0.0038654152303934097]
IN	IN	NL-HaNA_1.04.02_1072_0182.jpg	voornemen van den Goum: Generel; tegens Banda.; pa	[0.006870764307677746, 0.9862992167472839, 0.0032101941760629416, 0.003619859926402569]
IN	IN	NL-HaNA_1.04.02_1072_0183.jpg	vermeerden & versterckt.; met weymis costen voorde	[0.006249531172215939, 0.9864282011985779, 0.0036167390644550323, 0.0037054994609206915]
IN	IN	NL-HaNA_1.04.02_1072_0184.jpg	voor gamnisoeren.; Langewers dierstus tot tochten.	[0.00841083936393261, 0.97711

 95%|█████████▌| 20/21 [00:23<00:00,  1.78batch/s]

OUT	OUT	NL-HaNA_1.04.02_1547_0097.jpg		[0.0024918601848185062, 0.010693737305700779, 0.002333863405510783, 0.9844805598258972]
OUT	OUT	NL-HaNA_1.04.02_1547_0098.jpg		[0.001568082720041275, 0.00829284731298685, 0.0018177498131990433, 0.9883213639259338]
BEGIN	BEGIN	NL-HaNA_1.04.02_1547_0099.jpg	uwe; Hoog Edele agtb: gebiedende Heere en Heeren; 	[0.995026171207428, 0.002068228553980589, 0.00045953557128086686, 0.0024460754357278347]
IN	IN	NL-HaNA_1.04.02_1547_0100.jpg	heen mne  dn d n n lel een w en de e sou co  an d;	[0.00914937723428011, 0.9708470702171326, 0.004106663167476654, 0.01589682325720787]
IN	IN	NL-HaNA_1.04.02_1547_0101.jpg	te Nambedle, den prince Cartadavile, en Moerianato	[0.006321579217910767, 0.9867390394210815, 0.0029649375937879086, 0.003974375780671835]
IN	IN	NL-HaNA_1.04.02_1547_0102.jpg	werden; war op de ehien anmandijn lae telkens tot 	[0.004644239787012339, 0.9876202940940857, 0.003116086358204484, 0.004619344137609005]
IN	IN	NL-HaNA_1.04.02_1547_0103.jpg	41; Mall

100%|██████████| 21/21 [00:27<00:00,  1.33s/batch]

BEGIN	BEGIN	NL-HaNA_1.04.02_1490_0583.jpg	Copije Secrete Resolutien; genomen bij de Ho: Rege	[0.9973623156547546, 0.0015661944635212421, 0.00033882528077811003, 0.0007326914346776903]
IN	IN	NL-HaNA_1.04.02_1490_0584.jpg	in presentie; van zijn Ed. t te; volbrengen, soo s	[0.010986301116645336, 0.9798687100410461, 0.003987658303231001, 0.005157343577593565]
IN	IN	NL-HaNA_1.04.02_1490_0585.jpg	dat 's Comp. s dienaren en wel voorna„; mentlijk p	[0.007038549054414034, 0.985831081867218, 0.0032854059245437384, 0.0038450032006949186]
IN	IN	NL-HaNA_1.04.02_1490_0586.jpg	bij ons te boeck staen voor niet wel; geintentione	[0.006331078242510557, 0.9866373538970947, 0.0031721466220915318, 0.003859366523101926]
IN	IN	NL-HaNA_1.04.02_1490_0587.jpg	goetgevonden dat dese ontbiedinge; bij ons gemeen 	[0.006249660160392523, 0.986785352230072, 0.003204730339348316, 0.003760277759283781]
IN	IN	NL-HaNA_1.04.02_1490_0588.jpg	schaap harder herwaerts aen te doen; overkomen, ge	[0.006481838878244162, 0.9867219

In [20]:
training_data._class_counts()

Counter({<Label.IN: 1>: 1642,
         <Label.BEGIN: 0>: 83,
         <Label.END: 2>: 80,
         <Label.OUT: 3>: 68})

In [21]:
writer = csv.DictWriter(
    sys.stdout,
    fieldnames=["Metric"] + [label.name for label in Label],
    delimiter="\t",
)
writer.writeheader()

for metric in (precision, recall, f1_score):
    scores = {
        label.name: f"{score:.4f}"
        for label, score in zip(Label, metric.compute().tolist())
    }
    writer.writerow({"Metric": metric.__class__.__name__} | scores)

print(f"Accuracy ({accuracy.average} average):\t{accuracy.compute().item():.4f}")

Metric	BEGIN	IN	END	OUT
MulticlassPrecision	1.0000	1.0000	1.0000	1.0000
MulticlassRecall	1.0000	1.0000	1.0000	1.0000
MulticlassF1Score	1.0000	1.0000	1.0000	1.0000
Accuracy (micro average):	1.0000
